In [2]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

In [3]:
TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)

In [3]:
data = {"description":"i hate this stuff"}
headers= {'Content-type':"application/json",
           "cache-control":"no-cache"
          }
data= json.dumps(data)


In [44]:
%%time
r = requests.post("http://35.192.71.216:5000/", data = data, headers = headers)

Wall time: 1.55 s


In [43]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

def bert_request(tweet_text):
    data = {"description":tweet_text}
    headers= {'Content-type':"application/json",
               "cache-control":"no-cache"
              }
    data= json.dumps(data)
    return requests.post("http://35.192.71.216:5000/", data = data, headers = headers)

def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    #make api request for toxicity analysis
    bert_result = bert_request(tweet).json()
    
    tweet_info = {"tweet": 
                     {"user_id": full_tweet.user.id,
                      "user_name" : full_tweet.user.name,
                      "tweet": full_tweet.full_text,
                      "tweet_id" : full_tweet.id_str,
                     "bert_output": bert_result,
                     "tweet":tweet}
                    }
    return tweet_info
    

def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=20,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
        
def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers['content-type']
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == 'application/json': 
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN' in request_json:
            TWITTER_ACCESS_TOKEN = request_json['TWITTER_ACCESS_TOKEN']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN_SECRET' in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json['TWITTER_ACCESS_TOKEN_SECRET']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")
        
        # Call the function for the POST request. 
        if request.method == 'POST':
            return clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

## TEST

In [51]:
%%time
u = clean_timeline(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

Wall time: 31.8 s


In [52]:
u = json.loads(u)

In [37]:
home_timeline = TWITTER.home_timeline(count=20,
                                         tweet_mode='extended',
                                         exlude_rts=False)

In [44]:
process_tweet(home_timeline[0])

{'tweet': {'user_id': 50771088,
  'user_name': 'Mick Shaffer',
  'tweet': "Owings striking out against a position player isn't going to help his rep. I'm a journalist so I can sniff out stuff like that.",
  'tweet_id': '1125955943823544320',
  'bert_output': {'results': {'identity_hate': 0.00013447334640659392,
    'insult': 0.00019758346024900675,
    'obscene': 0.00015612311835866421,
    'severe_toxic': 0.0001237120886798948,
    'threat': 9.398273687111214e-05,
    'toxic': 0.0003531617112457752}}}}

In [53]:
headers = {"Content-Type":"application/json"}
body = {"TWITTER_ACCESS_TOKEN": config('TWITTER_ACCESS_TOKEN'),
        "TWITTER_ACCESS_TOKEN_SECRET": config('TWITTER_ACCESS_TOKEN_SECRET')
        }

In [54]:
%%time
r = requests.post("https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
                  headers=headers,
                  data=json.dumps(body))

Wall time: 30.8 s


In [55]:
r2 = r.json()

In [56]:
r2[0]

{'tweet': {'user_id': 23909137,
  'user_name': 'Larry Brown',
  'tweet': 'Drake, Raptors fans troll Joel Embiid with airplane in Game 5 https://t.co/hjh8KFHR4z',
  'tweet_id': '1125959710362849281',
  'bert_output': {'results': {'identity_hate': 0.00010662349814083427,
    'insult': 0.0002195992274209857,
    'obscene': 0.00018712588644120842,
    'severe_toxic': 0.00010429355461383238,
    'threat': 7.004194776527584e-05,
    'toxic': 0.0005336005124263465}}}}

In [57]:
[r2[i]["tweet"] for i in range(len(r2))]

[{'user_id': 23909137,
  'user_name': 'Larry Brown',
  'tweet': 'Drake, Raptors fans troll Joel Embiid with airplane in Game 5 https://t.co/hjh8KFHR4z',
  'tweet_id': '1125959710362849281',
  'bert_output': {'results': {'identity_hate': 0.00010662349814083427,
    'insult': 0.0002195992274209857,
    'obscene': 0.00018712588644120842,
    'severe_toxic': 0.00010429355461383238,
    'threat': 7.004194776527584e-05,
    'toxic': 0.0005336005124263465}}},
 {'user_id': 8496762,
  'user_name': 'tylercowen',
  'tweet': 'RT @pamela_herd: While Trump should release his tax returns, I am not thrilled that the NYT used de-identified publicly released data to do…',
  'tweet_id': '1125959002880450560',
  'bert_output': {'results': {'identity_hate': 0.00016027066158130765,
    'insult': 0.00021916520199738443,
    'obscene': 0.0001547282445244491,
    'severe_toxic': 0.0001458178012399003,
    'threat': 0.00011751330021070316,
    'toxic': 0.00033231236739084125}}},
 {'user_id': 16396078,
  'user_n